In [14]:
import numpy as np

<h2>[Problem 1] Classifying fully connected layers</h2>

In [15]:
class FC:
    """
    Number of nodes Fully connected layer from n_nodes1 to n_nodes2
    Parameters
    ----------
    n_nodes1 : int
      Number of nodes in the previous layer
    n_nodes2 : int
      Number of nodes in the later layer
    initializer: instance of initialization method
    optimizer: instance of optimization method
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        # Initialize
        # Initialize self.W and self.B using the initializer method
        self.W = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2)

    def forward(self, X):
        """
        forward
        Parameters
        ----------
        X : The following forms of ndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : The following forms of ndarray, shape (batch_size, n_nodes2)
            output
        """       
        self.X = X
        A = X @ self.W + self.B
        return A

    def backward(self, dA):
        """
        Backward
        Parameters
        ----------
        dA : The following forms of ndarray, shape (batch_size, n_nodes2)
            Gradient flowing from behind
        Returns
        ----------
        dZ : The following forms of ndarray, shape (batch_size, n_nodes1)
            Gradient to flow forward
        """
        self.dA = dA
        dZ = dA @ self.W.T
        # update
        self = self.optimizer.update(self)
        return dZ

<h2>[Problem 2] Classifying the initialization method</h2>

In [16]:
class SimpleInitializer:
    """
    Simple initialization with Gaussian distribution
    Parameters
    ----------
    sigma : float
      Standard deviation of Gaussian distribution
    """
    def __init__(self, sigma=0.01):
        self.sigma = sigma

    def W(self, n_nodes1, n_nodes2):
        """
        Weight initialization
        Parameters
        ----------
        n_nodes1 : int
          Number of nodes in the previous layer
        n_nodes2 : int
          Number of nodes in the later layer
        Returns
        ----------
        W : weight
        """
        return self.sigma * np.random.randn(n_nodes1, n_nodes2)

    def B(self, n_nodes2):
        """
        Bias initialization
        Parameters
        ----------
        n_nodes2 : int
          Number of nodes in the later layer
        Returns
        ----------
        B : bias
        """
        return self.sigma * np.random.randn(n_nodes2)

<h2>[Problem 3] Classifying optimization methods</h2>

In [17]:
class SGD:
    """
    Stochastic gradient descent
    Parameters
    ----------
    lr : Learning rate
    """
    def __init__(self, lr=0.01):
        self.lr = lr

    def update(self, layer):
        """
        Update weights and biases for a layer
        Parameters
        ----------
        layer : Instance of the layer before update
        """
        dW = layer.X.T @ layer.dA
        dB = np.sum(layer.dA, axis=0)
        layer.W -= self.lr * dW
        layer.B -= self.lr * dB
        return layer

<h2>[Problem 4] Classifying activation functions</h2>

In [18]:
class Tanh:
    def forward(self, A):
        """return Z"""
        return np.tanh(A)
    
    def backward(self, dZ, Z):
        """return dA"""
        return dZ * (1 - Z**2)

In [19]:
class Sigmoid:
    def forward(self, A):
        return 1 / (1 + np.exp(-A))
    
    def backward(self, dZ, Z):
        return dZ * ((1 - Z) * Z)

In [20]:
class Softmax:
    def forward(self, A):
        return np.exp(A) / np.sum(np.exp(A), axis=1).reshape(-1, 1)
    
    def backward(self, Z, y):
        """return dA, CEE"""
        return (Z - y) / y.shape[0], -np.sum((y * np.log(Z + 1e-7))) / y.shape[0]

<h2>[Problem 5] ReLU class creation</h2>

In [21]:
class ReLU:
    def forward(self, A):
        return np.maximum(A, 0)
    
    def backward(self, Z):
        return np.where(Z > 0, 1, 0)

<h2>[Problem 6] Initial value of weight</h2>

In [22]:
class XavierInitializer:
    def __init__(self, n_nodes1):
        self.sigma = 1.0 / n_nodes1**(1/2)

    def W(self, n_nodes1, n_nodes2):
        return self.sigma * np.random.randn(n_nodes1, n_nodes2)

    def B(self, n_nodes2):
        return self.sigma * np.random.randn(n_nodes2)

In [23]:
class HeInitializer:
    def __init__(self, n_nodes1):
        self.sigma = (2.0 / n_nodes1)**(1/2)

    def W(self, n_nodes1, n_nodes2):
        return self.sigma * np.random.randn(n_nodes1, n_nodes2)

    def B(self, n_nodes2):
        return self.sigma * np.random.randn(n_nodes2)

<h2>[Problem 7] Optimization method</h2>

In [24]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.H = 0

    def update(self, layer):
        dW = layer.X.T @ layer.dA
        self.H += dW**2
        layer.W -= (self.lr / self.H**(1/2)) * dW
        return layer

<h2>[Problem 8] Class completion</h2>

In [25]:
class GetMiniBatch:
    """
Iterator to get a mini-batch
    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random number seed
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop

    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]   

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [26]:
class ScratchDeepNeuralNetrowkClassifier():
    """
    Simple three-layer neural network classifier
    Parameters
    ----------
    Attributes
    ----------
    """
    def __init__(self, 
                hidden_layers=[400,200],
                hidden_activations=['tanh', 'tanh'],
                hidden_initializers=['gaussian', 'gaussian'],
                sigma=0.01,
                optimizer='sgd',
                layer_type=None,
                batch_size=20,
                epochs=10,
                lr=0.01,
                seed=0,
                verbose=False
                ):
        self.hidden_layers = hidden_layers
        self.activations_name = hidden_activations.copy()
        self.activations_name.insert(0, None)
        self.activations_name.append('softmax')
        self.initializers = hidden_activations.copy()
        self.initializers.insert(0, None)
        self.initializers.append('gaussian')
        self.sigma = sigma
        self.optimizer = optimizer
        self.layer_type = layer_type
        self.batch_size = batch_size
        self.epochs = epochs
        self.lr = lr
        self.seed = seed
        self.verbose = verbose

    def _initialize_neural_network(self):
        #choose optimizer 
        if (self.optimizer == 'sgd'):
            self.optimizer = SGD(self.lr)
        elif (self.optimizer == 'adagrad'):
            self.optimizer = AdaGrad(self.lr)
        #choose activations
        self.activations = np.full(len(self.n_nodes), None) #activations[0] unused
        for i in range(1, len(self.n_nodes)):
            if (self.activations_name[i] == 'tanh'):
                self.activations[i] = Tanh()
            elif (self.activations_name[i] == 'sigmoid'):
                self.activations[i] = Sigmoid()
            elif (self.activations_name[i] == 'relu'):
                self.activations[i] = ReLU()
            elif (self.activations_name[i] == 'softmax'):
                self.activations[i] = Softmax()
        #choose initializer and build neural network
        self.FC = np.full(len(self.n_nodes), None) #FC[0] unused
        for i in range(1, len(self.n_nodes)):
            if (self.initializers[i] == 'gaussian'):
                self.FC[i] = FC(self.n_nodes[i - 1], self.n_nodes[i], SimpleInitializer(self.sigma), self.optimizer)
            elif (self.initializers[i] == 'xavier'):
                self.FC[i] = FC(self.n_nodes[i - 1], self.n_nodes[i]. XavierInitializer(self.n_nodes[i - 1]), self.optimizer)
            elif (self.initializers[i] == 'he'):   
                self.FC[i] = FC(self.n_nodes[i - 1], self.n_nodes[i]. HeInitializer(self.n_nodes[i - 1]), self.optimizer)
        self.A = np.full(len(self.n_nodes), None) #A[0] unused
        self.Z = np.full(len(self.n_nodes), None)
        self.Z[0] = X
        self.dA = np.full(len(self.n_nodes), None) #A[0] unused
        self.dZ = np.full(len(self.n_nodes), None) #Z[-1] unused
        self.CEE_list = []

    def _forward(self, X):
        for i in range(1, len(self.n_nodes)):
            self.A[i] = self.FC[i].forward(self.Z[i - 1])
            self.Z[i] = self.activations[i].forward(self.A[i])

    def _forward_predict(self, X):
        Z_pred[0] = X
        for i in range(1, len(self.n_nodes)):
            A_pred[i] = self.FC[i].forward(Z_pred[i - 1])
            Z_pred[i] = self.activations[i].forward(A_pred[i])
        return Z_pred

    def _backward(self, y):
        for i in range(len(self.n_nodes) - 1, 0, -1):
            if (self.activations_name[i] == 'tanh' or self.activations_name[i] == 'sigmoid'):
                self.dA[i] = self.activations[i].backward(self.dZ, self.Z[i])
            elif (self.activations_name[i] == 'relu'):
                self.dA[i] = self.activations[i].backward(self.Z[i])
            elif (self.activations_name[i] == 'softmax'):
                self.dA[i], CEE = self.activations[i].backward(self.Z[i], y)
                self.CEE_list.append(CEE)
            self.dZ[i - 1] = self.FC[i].backward(self.dA[i])

    def _optimize(self, X, y):
        get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch_size, seed=self.seed)
        for _ in range(0, self.epochs):
            for mini_X, mini_y in get_mini_batch:
                self._forward(mini_X)
                self._backward(mini_y) 

    def fit(self, X, y, X_val=None, y_val=None):
        """
        Learn a neural network classifier.
        Parameters
        ----------
        X : The following forms of ndarray, shape (n_samples, n_features)
            Features of training data
        y : The following form of ndarray, shape (n_samples,)
            Correct answer value of training data
        X_val : The following forms of ndarray, shape (n_samples, n_features)
            Features of verification data
        y_val : The following form of ndarray, shape (n_samples,)
            Correct value of verification data
        """
        self.n_output = y.shape[1]
        self.n_features = X.shape[1]
        self.n_nodes = self.hidden_layers.copy()
        self.n_nodes.insert(0, self.n_features)
        self.n_nodes.append(self.n_output)
        self._initialize_neural_network()
        self._optimize(X, y)

    def predict(self, X):
        """
        Estimate using a neural network classifier.
        Parameters
        ----------
        X : The following forms of ndarray, shape (n_samples, n_features)
            sample
        Returns
        -------
            The following form of ndarray, shape (n_samples, 1)
            Estimated result
        """
        Z_pred = self._forward_predict(X)
        return np.argmax(Z_pred[-1], axis=1), Z_pred[-1]

<h2>[Problem 9] Learning and estimation</h2>